In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.circuit import ParameterVector
from qiskit import IBMQ, BasicAer, Aer, QuantumCircuit, execute
from qiskit.circuit.library import TwoLocal, EfficientSU2
from qiskit.quantum_info.states.random import random_statevector
from qiskit.quantum_info import state_fidelity
from qiskit.aqua.utils import get_subsystem_density_matrix
from qiskit.aqua import aqua_globals
from scipy.special import rel_entr
from qiskit.extensions import CXGate
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit.providers.aer.extensions.snapshot_statevector import *
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
# setup qiskit.chemistry logging
import logging
from qiskit.chemistry import set_qiskit_chemistry_logging
set_qiskit_chemistry_logging(logging.ERROR)

In [2]:
# Loading your IBM Q account(s)
backend = Aer.get_backend("qasm_simulator")
print(backend)

qasm_simulator


In [3]:
#Set the seed for reproducible results
aqua_globals.random_seed = 100
seed_simulator = 100
seed_transpiler=100
np.random.seed(100)

In [4]:
#Alternative way to attain the Haar distribution
def haar_distribution():
    nplist = np.load('haar_distribution.npy')
    flist = nplist.tolist()
    if len(flist) == 0:
        print('the list is empty')
        flist = []
        for m in range(5000):
            a = random_statevector(4)
            b = random_statevector(4)
            flist.append(state_fidelity(a, b))
        np.save('haar_distribution.npy', flist)
    return flist

In [5]:
#Determine expressibility for circuits with CZ and CX gates #
def quantum_state_preparation(parameters, gseq):
    for gorder, elist in gseq:

        circuit = QuantumCircuit(4)
        circuit.ry(parameters[0], 0)
        circuit.ry(parameters[1], 1)
        circuit.ry(parameters[2], 2)
        circuit.ry(parameters[3], 3)
        circuit.barrier()
        for i, j in enumerate(gorder):
            if gorder[i] == 'cx':
                circuit.cx(elist[i][0], elist[i][1])
            else:
                circuit.cz(elist[i][0], elist[i][1])
        circuit.barrier()
        circuit.ry(parameters[4], 0)
        circuit.ry(parameters[5], 1)
        circuit.ry(parameters[6], 2)
        circuit.ry(parameters[7], 3)
        circuit.snapshot_statevector('final')
        return circuit

def get_cxcz_distribution(ansatz,gates):
    cfidelityList = []
    for m in range(5000):
        qstatelist = []
        for i in range(2):
            # Obtain a quantum circuit instance from the paramters
            params = np.random.random(ansatz.num_parameters) * 2 * np.pi
            circ = quantum_state_preparation(params, gates)
            result = execute(circ, backend, seed_simulator=seed_simulator, seed_transpiler=seed_transpiler).result()
            outputstate = result.data(0)['snapshots']['statevector']['final']
            qstatelist.append(outputstate[0])
        cfidelityList.append(state_fidelity(qstatelist[0], qstatelist[1]))
    print(gates)
    circ.draw(output='mpl')
    return cfidelityList

In [6]:
#Compute KL divergence between distributions of state fidelities
def kl_divergence(p):
    N=16
    num_bins = 75
    bin_width = 1/num_bins
    bin_edges = np.linspace(0,1,num_bins+1)
    
    chist, _ =  np.histogram(np.array(p), bins=bin_edges, density= True)
    #Use analytical form to attain probability distribution of fidelities for ensemble of Haar random states
    fhist = np.array([(N-1)*(1-F)**(N-2) for F in bin_edges[:-1]])
    
    kl_pq = np.sum(rel_entr(chist*bin_width, fhist/sum(fhist)))
    print('KL(P || Q): %.3f ' % kl_pq)
    return kl_pq

In [7]:
num_qubits = 4
ansatz = TwoLocal(4, 'ry', 'cz', reps=1, entanglement=[(2, 3), (1, 2), (0,1), (0,3)], insert_barriers=True)

#CXCZ circuit experiment

gatelist = [(['cx', 'cz', 'cx', 'cz'], [(2, 3), (1, 2), (0, 1), (0, 3)]),
            (['cx', 'cz', 'cz', 'cx'], [(2, 3), (1, 2), (0, 1), (0, 3)]),
            (['cx', 'cx', 'cz', 'cz'], [(2, 3), (1, 2), (0, 1), (0, 3)]),
            (['cz', 'cx', 'cz', 'cx'], [(2, 3), (1, 2), (0, 1), (0, 3)]),
            (['cz', 'cx', 'cx', 'cz'], [(2, 3), (1, 2), (0, 1), (0, 3)]),
            (['cz', 'cz', 'cx', 'cx'], [(2, 3), (1, 2), (0, 1), (0, 3)])]

for i in range(len(gatelist)):
    gates = [gatelist[i]]
    cfidelityList = get_cxcz_distribution(ansatz, gates)
    kl = kl_divergence(cfidelityList)
    #visualise circuits
    #params = ParameterVector('θ', 8) 
    #circ = quantum_state_preparation(params, gates)
    #circ.draw(output='mpl')

[(['cx', 'cz', 'cx', 'cz'], [(2, 3), (1, 2), (0, 1), (0, 3)])]
KL(P || Q): 0.229 
[(['cx', 'cz', 'cz', 'cx'], [(2, 3), (1, 2), (0, 1), (0, 3)])]
KL(P || Q): 0.234 
[(['cx', 'cx', 'cz', 'cz'], [(2, 3), (1, 2), (0, 1), (0, 3)])]
KL(P || Q): 0.205 
[(['cz', 'cx', 'cz', 'cx'], [(2, 3), (1, 2), (0, 1), (0, 3)])]
KL(P || Q): 0.202 
[(['cz', 'cx', 'cx', 'cz'], [(2, 3), (1, 2), (0, 1), (0, 3)])]
KL(P || Q): 0.224 
[(['cz', 'cz', 'cx', 'cx'], [(2, 3), (1, 2), (0, 1), (0, 3)])]
KL(P || Q): 0.240 
